In [ ]:
import importlib.util

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

try:
    import quimb
except ImportError:
    print("installing cirq-core[contrib]...")
    !pip install --quiet 'cirq-core[contrib]'
    print("installed cirq-core[contrib].")

# Contract a Grid Circuit
Shallow circuits on a planar grid with low-weight observables permit easy contraction.

Note: this notebook relies on unreleased Cirq features. If you want to try these features, make sure you install cirq via `pip install cirq --pre`.

### Imports

In [ ]:
import numpy as np
import networkx as nx

import cirq
import quimb
import quimb.tensor as qtn
from cirq.contrib.svg import SVGCircuit

import cirq.contrib.quimb as ccq

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import seaborn as sns
sns.set_style('ticks')

plt.rc('axes', labelsize=16, titlesize=16)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14, title_fontsize=16)

In [ ]:
# theme colors
QBLUE = '#1967d2'
QRED = '#ea4335ff'
QGOLD = '#fbbc05ff'
QGREEN = '#34a853ff'

QGOLD2 = '#ffca28'
QBLUE2 = '#1e88e5'

## Make an example circuit topology
We'll use entangling gates according to this topology and compute the value of an observable on the red nodes.

In [ ]:
width = 3
height = 4
graph = nx.grid_2d_graph(width, height)
rs = np.random.RandomState(52)
nx.set_edge_attributes(graph, name='weight',
                       values={e: np.round(rs.uniform(), 2) for e in graph.edges})

zz_inds = ((width//2, (height//2-1)), (width//2, (height//2)))
nx.draw_networkx(graph, 
                 pos={n:n for n in graph.nodes},
                 node_color=[QRED if node in zz_inds else QBLUE for node in graph.nodes])

### Circuit

In [ ]:
qubits = [cirq.GridQubit(*n) for n in graph]
circuit = cirq.Circuit(
    cirq.H.on_each(qubits),
    ccq.get_grid_moments(graph),
    cirq.Moment([cirq.rx(0.456).on_each(qubits)]),
)
SVGCircuit(circuit)

### Observable

In [ ]:
ZZ = cirq.Z(cirq.GridQubit(*zz_inds[0])) * cirq.Z(cirq.GridQubit(*zz_inds[1]))
ZZ

### The contraction
The value of the observable is $\langle 0 | U^\dagger (ZZ) U |0 \rangle$.

In [ ]:
tot_c = ccq.circuit_for_expectation_value(circuit, ZZ)
SVGCircuit(tot_c)

## We can simplify the circuit
By cancelling the "forwards" and "backwards" part of the circuit that are outside of the light-cone of the observable, we can reduce the number of gates to consider --- and sometimes the number of qubits involved at all. To see this in action, run the following cell and then keep re-running the following cell to watch gates disappear from the circuit.

In [ ]:
compressed_c = tot_c.copy()
print(len(list(compressed_c.all_operations())), len(compressed_c.all_qubits()))

**(try re-running the following cell to watch the circuit get smaller)**

In [ ]:
compressed_c= cirq.merge_k_qubit_unitaries(compressed_c, k=2)
compressed_c = cirq.merge_k_qubit_unitaries(compressed_c, k=1)

compressed_c = cirq.drop_negligible_operations(compressed_c, atol=1e-6)
compressed_c = cirq.drop_empty_moments(compressed_c)
print(len(list(compressed_c.all_operations())), len(compressed_c.all_qubits()))
SVGCircuit(compressed_c)

### Utility function to fully-simplify

We provide this utility function to fully simplify a circuit.

In [ ]:
ccq.simplify_expectation_value_circuit(tot_c)
SVGCircuit(tot_c)

In [ ]:
# simplification might eliminate qubits entirely for large graphs and 
# shallow `p`, so re-get the current qubits.
qubits = sorted(tot_c.all_qubits())
print(len(qubits))

## Turn it into a Tensor Netowork

We explicitly "cap" the tensor network with `<0..0|` bras so the entire thing contracts to the expectation value $\langle 0 | U^\dagger (ZZ) U |0 \rangle$.

In [ ]:
tensors, qubit_frontier, fix = ccq.circuit_to_tensors(
    circuit=tot_c, qubits=qubits)
end_bras = [
    qtn.Tensor(
        data=quimb.up().squeeze(),
        inds=(f'i{qubit_frontier[q]}_q{q}',),
        tags={'Q0', 'bra0'}) for q in qubits
]

tn = qtn.TensorNetwork(tensors + end_bras)
tn.graph(color=['Q0', 'Q1', 'Q2'])
plt.show()

### `rank_simplify` effectively folds together 1- and 2-qubit gates

In practice, using this is faster than running the circuit optimizer to remove gates that cancel themselves, but please benchmark for your particular use case.

In [ ]:
tn.rank_simplify(inplace=True)
tn.graph(color=['Q0', 'Q1', 'Q2'])

### The tensor contraction path tells us how expensive this will be

In [ ]:
path_info = tn.contract(get='path-info')

In [ ]:
path_info.opt_cost / int(3e9) # assuming 3gflop, in seconds

In [ ]:
path_info.largest_intermediate * 128 / 8 / 1024 / 1024 / 1024 # gb

### Do the contraction

In [ ]:
zz = tn.contract(inplace=True)
zz = np.real_if_close(zz)
print(zz)

## Big Circuit

In [ ]:
width = 8
height = 8
graph = nx.grid_2d_graph(width, height)
rs = np.random.RandomState(52)
nx.set_edge_attributes(graph, name='weight',
                       values={e: np.round(rs.uniform(), 2) for e in graph.edges})

zz_inds = ((width//2, (height//2-1)), (width//2, (height//2)))
nx.draw_networkx(graph, 
                 pos={n:n for n in graph.nodes},
                 node_color=[QRED if node in zz_inds else QBLUE for node in graph.nodes])

In [ ]:
qubits = [cirq.GridQubit(*n) for n in graph]
circuit = cirq.Circuit(
    cirq.H.on_each(qubits),
    ccq.get_grid_moments(graph),
    cirq.Moment([cirq.rx(0.456).on_each(qubits)]),
)

In [ ]:
ZZ = cirq.Z(cirq.GridQubit(*zz_inds[0])) * cirq.Z(cirq.GridQubit(*zz_inds[1]))
ZZ

In [ ]:
ccq.tensor_expectation_value(circuit, ZZ)